In [58]:
import requests
import os
import json
import pandas as pd
import datetime

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# bearer_token = os.environ.get("BEARER_TOKEN")
bearer_token ='AAAAAAAAAAAAAAAAAAAAACqHUgEAAAAAcghJSkKBmFmw36ZiRxrqVdyomIk%3DZlDqMjNA0ka4btIuhDjiHUyRGoCHDzKBWvNGcqYXpLJ5kuhUAp'

In [59]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [80]:
def dataframe_from_tweets(keyword, from_date, to_date):
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    query_params = {
        'query': keyword, 
        'start_time': from_date.isoformat("T") + "Z", 
        'end_time': to_date.isoformat("T") + "Z",
        'tweet.fields': 'author_id,created_at,geo', 
        'max_results': 20}
    print(query_params)
    json_response = connect_to_endpoint(search_url, query_params)
    df = pd.DataFrame(json_response['data']) 
    # df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'), index=['x', 'y'])
    df['Keyword'] = keyword
    return df
    

In [81]:
class PlanItem:
    def __init__(self, year, month, day):
        self.from_date = datetime.datetime(year, month, day)
        self.to_date = self.from_date + datetime.timedelta(days=15)
    def file_name(self):
        return self.from_date.strftime('tweets_%Y%m%d.json')

In [82]:
plan = [
    PlanItem(2021,6,1),
    PlanItem(2021,6,15),
    PlanItem(2021,7,1),
    PlanItem(2021,7,15),
    PlanItem(2021,8,1),
    PlanItem(2021,8,15),
    PlanItem(2021,9,1),
    PlanItem(2021,9,15),
    PlanItem(2021,10,1),
    PlanItem(2021,10,15),
]
keywords = ['altcoin', 'bitcoin', 'coindesk', 'cryptocurrency', 'gold', 'APPL', 'GOOG', 'YHOO']

In [83]:
for item in plan:
    item_df = pd.DataFrame()
    for keyword in keywords:
        keyword_df = dataframe_from_tweets(keyword, item.from_date, item.to_date)
        item_df = item_df.append(keyword_df, ignore_index=True)
    print(f'Saving {item.file_name()}')
    item_df.to_json(f'data/{item.file_name()}')
        

{'query': 'altcoin', 'start_time': '2021-06-01T00:00:00Z', 'end_time': '2021-06-16T00:00:00Z', 'tweet.fields': 'author_id,created_at,geo', 'max_results': 20}
400


Exception: (400, '{"errors":[{"parameters":{"start_time":["2021-06-01T00:00Z"]},"message":"Invalid \'start_time\':\'2021-06-01T00:00Z\'. \'start_time\' must be on or after 2021-10-13T02:29Z"},{"parameters":{"end_time":["2021-06-16T00:00Z"]},"message":"Invalid \'end_time\':\'2021-06-16T00:00Z\'. \'end_time\' must be on or after 2021-10-13T02:29Z"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}')